<a href="https://colab.research.google.com/github/rerebebeccacca/20210624-deep-learning-class/blob/main/05_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# download dataset from https://drive.google.com/file/d/12iinuMZkaZOZGXlkPsUnmETaE-cMlyHu/view?usp=sharing
!gdown --id 12iinuMZkaZOZGXlkPsUnmETaE-cMlyHu --output defect.zip

Downloading...
From: https://drive.google.com/uc?id=12iinuMZkaZOZGXlkPsUnmETaE-cMlyHu
To: /content/defect.zip
53.2MB [00:00, 94.1MB/s]


In [6]:
# unzip file
!unzip -q defect.zip

replace __MACOSX/._Magnetic-tile-defect? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/._MT_Uneven? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/._MT_Break? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Magnetic-tile-defect/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/._MT_Blowhole? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/._MT_Fray? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/._MT_Crack? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/._MT_Free? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/MT_Uneven/._Imgs? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Magnetic-tile-defect/MT_Break/._Imgs? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Magneti

In [18]:
import tensorflow as tf
import cv2 #讀圖套件
import numpy as np
from glob import glob#去搜尋的套件
from sklearn.model_selection import train_test_split

In [19]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, img_paths, batch_size, img_size, shuffle=True, aug=False):
        self.img_paths = img_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.aug = aug

        self.indexes = np.arange(len(self.img_paths)) #要去第幾個位置去抓圖片 [0,1,2,3,4,5]
        self.on_epoch_end()

    #看每個epoch有幾個batches
    def __len__(self):
        return int(np.ceil(len(self.img_paths) / self.batch_size)) # batches per epoch

    def __getitem__(self, index):
        # Generate indexes of the batch
        idxs = self.indexes[index * self.batch_size:(index + 1) * self.batch_size] #看要去哪個位置去取得圖片
        # Find list of IDs
        batch_img_paths = [self.img_paths[i] for i in idxs]

        # Generates data containing batch_size samples
        x = np.empty((len(batch_img_paths), self.img_size, self.img_size, 3), dtype=np.float32)#第一維看有幾張
        y = np.empty((len(batch_img_paths), 1), dtype=np.float32)#y是向量，看有幾筆資料，及類別數值

        for i, img_path in enumerate(batch_img_paths):
            # read img
            img = cv2.imread(img_path)
            img = cv2.resize(img, (self.img_size, self.img_size))
            img = img / 255.

            if self.aug:
                img = img # Augmentation
                
            # read class label
            cls = img_path.split('/')[-3]
            if cls == 'MT_Free':
                cls = 0
            else:
                cls = 1
            x[i] = img
            y[i] = cls
        return x, y

    def on_epoch_end(self):
        # Updates indexes after each epoch
        if self.shuffle: #把資料順序打亂，增加隨機性
            np.random.shuffle(self.indexes)

In [20]:
#設定圖片路徑，去搜尋符合自訂的圖片檔案路徑下所有jpg的檔
img_paths = glob('./Magnetic-tile-defect/MT_Free/Imgs/*.jpg') + glob('./Magnetic-tile-defect/MT_Blowhole/Imgs/*.jpg')

In [21]:
len(img_paths)

1067

In [22]:
# 切成8:2的比例
train_img_paths, val_img_paths = train_test_split(img_paths, test_size=0.2)

In [23]:
#def __len__()
len(train_img_paths), len(val_img_paths)

(853, 214)

In [24]:

train_gen = DataGenerator(train_img_paths, 32, 200, shuffle=True, aug=True)
val_gen = DataGenerator(val_img_paths, 32, 200, shuffle=False, aug=False) #test時，要關掉shuffle=False, aug=False

In [25]:

# get 1 batch
# def __getitem__() 因為有做這個
imgs, labels = train_gen[2] #x = img ,y = labels

In [26]:
imgs.shape, labels.shape

((32, 200, 200, 3), (32, 1))

In [27]:
print(labels)

[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]]


In [31]:
# training

model.fit(train_gen,
          validation_data=val_gen,
          epochs=10,
          )

NameError: ignored